In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
np.random.seed(156)

In [4]:
trn = pd.read_csv('train_ver2.csv')
tst = pd.read_csv('test_ver2.csv')

/Users/yshin/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/yshin/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


**데이터 전처리** :

- 제품 변수의 결측값을 0으로 대체 (정보 없음 = 해당 제품 미보유)
- 훈련/테스트 데이터 통합 *테스트 데이터에 없는 제품 변수는 0
- 범주형 데이터는 .factorize() 통해 label encoding
- 수치형 데이터는 .unique()를 통해 특이값 처리, 정수형으로 변환
- 학습에 사용할 변수 이름을 features 리스트로 저장

In [5]:
# 제품 변수를 별도로 저장해 놓는다.
prods = trn.columns[24:].tolist()

# 제품 변수 결측값을 미리 0으로 대체
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# 24개 제품 중 하나도 보유하지 않는 고객 데이터는 제거
no_product = trn[prods].sum(axis=1) == 0
trn = trn[~no_product]

In [6]:
# 훈련 데이터와 테스트 데이터를 통합
for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn, tst], axis=0)

In [7]:
features = [] #학습에 사용할 변수 리스트

## 범주형 변수: label encoding
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_cols

# 수치형 변수: 특이값과 결측값을 -99로 대체, 정수형으로 변환
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

**피쳐 엔지니어링** :
- 24개의 고객 변수와 4개의 날짜 변수 기반 파생 변수, 24개의 lag-1 변수 사용
- 고객이 첫 계약 맺은 날짜(fecha_alta)와 고객이 마지막으로 1등급이었던 날짜(ult_fec_cli_lt)에서 연도와 월 정보 추출
- 결측값은 임시로 -99로 대체
- lag-N 변수는 N개월 전 보유 여부를 나타냄

In [8]:
# 두 날짜 변수에서 연도와 월 정보를 추출
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']
df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

In [9]:
#결측 처리
df.fillna(-99, inplace=True)

In [10]:
### lag-1 데이터 생성하기

# 날짜를 숫자로 변환: 2015-01-28은 1, 2016-06-28은 18로
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] 
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns ]
df_lag['int_date'] += 1

# 원본 데이터와 lag 데이터를 ncodper와 int_date 기준으로 통합
df_trn = df.merge(df_lag, on=['ncodpers','int_date'], how='left')
del df, df_lag #불필요한 데이터 제거

# 저번 달의 제품 정보가 존재하지 않을 경우 0으로 대체
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)
df_trn.fillna(-99, inplace=True)

# 변수 리스트에 추가
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]

**모델 학습** :
- 교차검증: 그전 18개월 데이터가 주어지고 미래의 201606 데이터를 예측해야 하므로, 최신 데이터(201605)를 검증 데이터로 분리하고 나머지를 훈련으로 사용하는 것이 일반적, 일단 201601~201604 데이터를 훈련으로 사용. 평가척도인 MAP@7.
- XGBoost
- 검증데이터의 실제 정답값을 기반으로 MAP@7 계산하면 0.042663(최대값. 모든 고객이 신규구매를 하지 않았으므로)-이를 감안하여 평가하기

In [11]:
use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']
del df_trn

# 훈련 데이터에서 신규 구매 건수만 추출
X = []
Y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i
    X.append(prX)
    Y.append(prY)
XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

# 훈련, 검증 데이터로 분리
vld_date = '2016-05-28'
XY_trn = XY[XY['fecha_dato'] != vld_date]
XY_vld = XY[XY['fecha_dato'] == vld_date]

In [12]:
# 훈련, 검증 데이터를 XGBoost 형태로 변환
X_trn = XY_trn.as_matrix(columns=features)
Y_trn = XY_trn.as_matrix(columns=['y'])
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

X_vld = XY_vld.as_matrix(columns=features)
Y_vld = XY_vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

param = {
    'booster': 'gbtree',
    'max_depth': 8,
    'nthread': 4,
    'num_class': len(prods),
    'objective': 'multi:softprob',
    'silent': 1,
    'eval_metric': 'mlogloss',
    'eta': 0.1,
    'min_child_weight': 10,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'seed': 2018,
    }

/Users/yshin/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/Users/yshin/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/yshin/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/Users/yshin/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [13]:
#훈련 데이터 학습
watch_list = [(dtrn, 'train'), (dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20)

[0]	train-mlogloss:2.70304	eval-mlogloss:2.71262
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:2.46704	eval-mlogloss:2.47922
[2]	train-mlogloss:2.27876	eval-mlogloss:2.29281
[3]	train-mlogloss:2.14652	eval-mlogloss:2.16172
[4]	train-mlogloss:2.03258	eval-mlogloss:2.0492
[5]	train-mlogloss:1.9349	eval-mlogloss:1.95236
[6]	train-mlogloss:1.85608	eval-mlogloss:1.87393
[7]	train-mlogloss:1.78396	eval-mlogloss:1.80199
[8]	train-mlogloss:1.72348	eval-mlogloss:1.74199
[9]	train-mlogloss:1.66871	eval-mlogloss:1.68748
[10]	train-mlogloss:1.62359	eval-mlogloss:1.64315
[11]	train-mlogloss:1.58044	eval-mlogloss:1.60003
[12]	train-mlogloss:1.54032	eval-mlogloss:1.55999
[13]	train-mlogloss:1.50486	eval-mlogloss:1.52505
[14]	train-mlogloss:1.47283	eval-mlogloss:1.49331
[15]	train-mlogloss:1.44302	eval-mlogloss:1.46372
[16]	train-mlogloss:1.41661	eval-mlogloss:1.43781
[17]	train-

[161]	train-mlogloss:0.999602	eval-mlogloss:1.08823
[162]	train-mlogloss:0.999053	eval-mlogloss:1.08823
[163]	train-mlogloss:0.998557	eval-mlogloss:1.08825
[164]	train-mlogloss:0.997948	eval-mlogloss:1.08826
[165]	train-mlogloss:0.997319	eval-mlogloss:1.08823
[166]	train-mlogloss:0.996664	eval-mlogloss:1.08816
[167]	train-mlogloss:0.996206	eval-mlogloss:1.08819
[168]	train-mlogloss:0.995539	eval-mlogloss:1.08819
[169]	train-mlogloss:0.994933	eval-mlogloss:1.0882
[170]	train-mlogloss:0.994335	eval-mlogloss:1.08821
[171]	train-mlogloss:0.993719	eval-mlogloss:1.0882
[172]	train-mlogloss:0.993045	eval-mlogloss:1.08824
[173]	train-mlogloss:0.992507	eval-mlogloss:1.08823
[174]	train-mlogloss:0.991895	eval-mlogloss:1.08815
[175]	train-mlogloss:0.991339	eval-mlogloss:1.08813
[176]	train-mlogloss:0.990784	eval-mlogloss:1.08815
[177]	train-mlogloss:0.990279	eval-mlogloss:1.08815
[178]	train-mlogloss:0.989619	eval-mlogloss:1.08812
[179]	train-mlogloss:0.989198	eval-mlogloss:1.08815
[180]	train-ml

In [16]:
#모델 저장
best_ntree_limit = model.best_ntree_limit

In [17]:
# 고객 식별 번호를 추출
vld = trn[trn['fecha_dato'] == vld_date]
ncodpers_vld = vld.as_matrix(columns=['ncodpers'])
# 검증 데이터에서 신규 구매
for prod in prods:
    prev = prod + '_prev'
    padd = prod + '_add'
    vld[padd] = vld[prod] - vld[prev]    
add_vld = vld.as_matrix(columns=[prod + '_add' for prod in prods])
add_vld_list = [list() for i in range(len(ncodpers_vld))]

# 고객별 신규 구매 정답 값을 add_vld_list에, 총 count를 count_vld에 저장
count_vld = 0
for ncodper in range(len(ncodpers_vld)):
    for prod in range(len(prods)):
        if add_vld[ncodper, prod] > 0:
            add_vld_list[ncodper].append(prod)
            count_vld += 1

/Users/yshin/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/yshin/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/yshin/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


In [19]:
#MAP@7
def apk(actual, predicted, k=7, default=0.0):
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    # 정답값이 공백일 경우, 무조건 0.0점을 반환
    if not actual:
        return default
    return score / min(len(actual), k)

def mapk(actual, predicted, k=7, default=0.0):
    # list of list인 정답값(actual)과 예측값(predicted)에서 고객별 Average Precision을 구하고, np.mean()을 통해 평균을 계산
    return np.mean([apk(a, p, k, default) for a, p in zip(actual, predicted)]) 

# 검증 데이터에서 얻을 수 있는 MAP@7 최고점
print(mapk(add_vld_list, add_vld_list, 7, 0.0))

0.04266379915553903


In [20]:
# 전체 훈련 데이터
X_all = XY.as_matrix(columns=features)
Y_all = XY.as_matrix(columns=['y'])
dall = xgb.DMatrix(X_all, label=Y_all, feature_names=features)
watch_list = [(dall, 'train')]
# 트리 개수를 늘어난 데이터 양만큼 비례해서 증가
best_ntree_limit = int(best_ntree_limit * (len(XY_trn) + len(XY_vld)) / len(XY_trn))
# 재학습!
model = xgb.train(param, dall, num_boost_round=best_ntree_limit, evals=watch_list)

/Users/yshin/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/Users/yshin/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


[0]	train-mlogloss:2.6967
[1]	train-mlogloss:2.44469
[2]	train-mlogloss:2.27264
[3]	train-mlogloss:2.1345
[4]	train-mlogloss:2.02023
[5]	train-mlogloss:1.92685
[6]	train-mlogloss:1.84516
[7]	train-mlogloss:1.77828
[8]	train-mlogloss:1.71895
[9]	train-mlogloss:1.66587
[10]	train-mlogloss:1.61836
[11]	train-mlogloss:1.57639
[12]	train-mlogloss:1.53857
[13]	train-mlogloss:1.50259
[14]	train-mlogloss:1.47056
[15]	train-mlogloss:1.44218
[16]	train-mlogloss:1.41604
[17]	train-mlogloss:1.39154
[18]	train-mlogloss:1.36921
[19]	train-mlogloss:1.34925
[20]	train-mlogloss:1.33049
[21]	train-mlogloss:1.31304
[22]	train-mlogloss:1.29741
[23]	train-mlogloss:1.28297
[24]	train-mlogloss:1.26929
[25]	train-mlogloss:1.25651
[26]	train-mlogloss:1.24486
[27]	train-mlogloss:1.23378
[28]	train-mlogloss:1.22347
[29]	train-mlogloss:1.21412
[30]	train-mlogloss:1.20526
[31]	train-mlogloss:1.1969
[32]	train-mlogloss:1.18917
[33]	train-mlogloss:1.18208
[34]	train-mlogloss:1.1753
[35]	train-mlogloss:1.16866
[36]	t

[284]	train-mlogloss:0.952272
[285]	train-mlogloss:0.951937
[286]	train-mlogloss:0.951556
[287]	train-mlogloss:0.951189
[288]	train-mlogloss:0.950769


In [ ]:
# 테스트 데이터 예측
X_tst = tst.as_matrix(columns=features)
dtst = xgb.DMatrix(X_tst, feature_names=features)
preds_tst = model.predict(dtst, ntree_limit=best_ntree_limit)
ncodpers_tst = tst.as_matrix(columns=['ncodpers'])
preds_tst = preds_tst - tst.as_matrix(columns=[prod + '_prev' for prod in prods])

/Users/yshin/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
